In [66]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB6 --baud=115200 
Ready.


In [59]:
%sendtofile config.txt

wifiname        DoESLiverpool
wifipassword    decafbad00
mqttbroker      10.0.100.1
pinled          2
boardname       miniwheels1
wifialt         DoESLiverpool,decafbad00,10.0.100.1
mAen            13
mAfore          27
mAback          26
mBen            12
mBfore          33
mBback          32


Sent 12 lines (296 bytes) to config.txt.


In [62]:
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py


Sent 626 lines (22505 bytes) to mqtt_as.py.


In [67]:
%sendtofile main.py

from mqtt_as import MQTTClient, config
import uasyncio as asyncio
import os, time, machine
from machine import Pin, PWM

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)

mApwm = PWM(Pin(int(fconfig["mAen"])))
mAfore = Pin(int(fconfig["mAfore"]), Pin.OUT)
mAback = Pin(int(fconfig["mAback"]), Pin.OUT)
mApwm.freq(10)
mApwm.duty(0)

mBpwm = PWM(Pin(int(fconfig["mBen"])))
mBfore = Pin(int(fconfig["mBfore"]), Pin.OUT)
mBback = Pin(int(fconfig["mBback"]), Pin.OUT)
mBpwm.freq(10)
mBpwm.duty(0)

timeZeroBy = 1
async def zeroingtask():
    global timeZeroBy
    while True:
        if timeZeroBy != 0:
            if time.ticks_ms() > timeZeroBy:
                timeZeroBy = 0
                mAfore.value(0)
                mAback.value(0)
                mBfore.value(0)
                mBback.value(0)
                pinled.value(0)
        await asyncio.sleep_ms(50)


def updatewheels(msg):
    global timeZeroBy
    try:
        s = msg.split()
        timeZeroBy = time.ticks_ms() + int(s[6])
        mApwm.duty(int(s[0]))
        mAfore.value(int(s[1]))
        mAback.value(int(s[2]))
        mBpwm.duty(int(s[3]))
        mBfore.value(int(s[4]))
        mBback.value(int(s[5]))
        pinled.value(1)
    except Exception as e:
        print("Bad wheels", msg, e)


boardname = fconfig["boardname"]
tboardcmd = boardname.encode() + b"/cmd"
tboardwheels = boardname.encode() + b"/wheels"

for l in ["500 1 0 500 1 0 1", "500 0 1 500 0 1 1", 
          "500 1 0 500 0 1 1", "500 0 1 500 1 0 1", "500 0 0 500 0 0 1"]:
    updatewheels(l)
    time.sleep_ms(500)

async def callbackcmdtask(cmd):
    print("executing", [cmd])
    pinled.value(1)
    try:
        exec(cmd)
        await client.publish(boardname+'/reply', "1")
    except Exception as e:
        print(e)
        await client.publish(boardname+'/reply/exception', str(e))
    pinled.value(0)

def callbackcmd(topic, msg, retained):
    if topic == tboardcmd:
        aloop.create_task(callbackcmdtask(msg))
    else:
        updatewheels(msg)
        
async def onconnecttask(client):
    print("subscribing")
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(boardname+'/ip', ipnumber, retain=True, qos=1)
    print("subscribing to", tboardcmd, tboardwheels)
    await client.subscribe(tboardcmd, 1)
    await client.subscribe(tboardwheels, 1)
    
async def mqtttask(bflip=False):
    print("mqttttask", bflip)
    if bflip and "wifialt" in fconfig:
        assid, awifi_pw, aserver = fconfig["wifialt"].split(",")
        if client._ssid == assid:
            assid, awifi_pw, aserver = fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"]
        client._ssid, client._wifi_pw, client.server = assid, awifi_pw, aserver
    try:
        await client.connect()
        return
    except OSError as e:
        print("client connect fail", [e])
    aloop.create_task(mqtttask(True))
        
        
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]

MQTTClient.DEBUG = True
client = MQTTClient(config)

aloop = asyncio.get_event_loop()
aloop.create_task(mqtttask())
aloop.create_task(zeroingtask())
aloop.run_forever()


Sent 114 lines (3353 bytes) to main.py.


In [68]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.


In [86]:

updatewheels("500 0 0 500 0 0 1")

In [80]:
mAfore.value(0)
mAback.value(0)
mBfore.value(0)
mBback.value(0)
